In [1]:
# 預測鐵達尼號生還者

import pandas as pd
train_df = pd.read_csv("Titanic_train.csv", encoding="utf-8")
test_df = pd.read_csv("Titanic_test.csv", encoding="utf-8")

In [2]:
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


# 要預測418筆資料
## 先抓特徵
1(數字).Age, SibSp, Parch ,Fare<br />
2(類別).PClass(艙等), Name(稱謂), Sex, Embarked

## 把缺失值補起來
sklearn的規定啦<br />
1(數字).<br />
<br />
中位數<br />
平均<br />
<br />
<br />
2(類別).補最可能的值

In [3]:
# 看缺失的欄位

train_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
# 看中位數

train_df.median()

# Pclass不可以填中位數，因為它是指艙等
# 解決方法：把1, 2, 3轉成字串
# 但是這裡沒缺資料所以不理它

PassengerId    446.0000
Survived         0.0000
Pclass           3.0000
Age             28.0000
SibSp            0.0000
Parch            0.0000
Fare            14.4542
dtype: float64

In [5]:
# 把缺的值都填成中位數，這邊只補數字類型的

med = train_df.median()
train_df = train_df.fillna(med)
test_df = test_df.fillna(med)

In [6]:
train_df.isna().sum()

# 可以看到"age"的缺已經被補起來了

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [7]:
# 這邊補文字類型的

most = train_df["Embarked"].value_counts().idxmax()
train_df["Embarked"] = train_df["Embarked"].fillna(most)
test_df["Embarked"] = test_df["Embarked"].fillna(most)

# Embarked(登船口) 有 S, C, Q三種，其中最多的是 S

In [8]:
train_df.isna().sum()

# 可以看到"Embarked"已經被換掉了

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

# 把非數值類別的資料，轉成是非題
用Embarked(登船口)舉例<br />
S、C、Q 演算法會把他排成0、1、2<br />
這樣會有大小數據，會影響演算法的判斷<br />
所以要把他轉成<br />

Embarked | S | C | Q
---- | --- | --- | ---
S | 1 | 0 | 0
C | 0 | 1 | 0
Q | 0 | 0 | 1


### 但是有些類別可以偷懶

1.類別本身有大小關係的<br />
比如說"Pclass(艙等)"<br />
<br />
2.類別只有兩種分類的

In [9]:
# 想辦法把中間的稱謂拿出來

n = "Braund, Mr. Owen Harris"
n = n.split(",")[-1].split(".")[0]
n.strip()

'Mr'

In [10]:
# 訓練資料的稱謂被拿出來了

def nameflow(n):
    n = n.split(",")[-1].split(".")[0]
    return n.strip()
train_df["Name"].apply(nameflow).value_counts()

Mr              517
Miss            182
Mrs             125
Master           40
Dr                7
Rev               6
Col               2
Mlle              2
Major             2
the Countess      1
Capt              1
Sir               1
Don               1
Ms                1
Lady              1
Mme               1
Jonkheer          1
Name: Name, dtype: int64

In [11]:
# 把結果小於30的排除

mid = train_df["Name"].apply(nameflow).value_counts()
reserved = mid[mid > 30].index
reserved

Index(['Mr', 'Miss', 'Mrs', 'Master'], dtype='object')

In [12]:
# 把練資料跟測試資料的"Name"都處理一下
# 並且把排除在外的設成"None"

def nameflow2(n):
    n = n.split(",")[-1].split(".")[0]
    n = n.strip    
    if n in reserved:
        return n
    else:
        return None
train_df["Name"] = train_df["Name"].apply(nameflow2)
test_df["Name"] = test_df["Name"].apply(nameflow2)

In [13]:
# dummies 把空白的補成其他

pd.get_dummies(train_df, columns=["Name", "Sex", "Embarked"], dummy_na=True)

# 可以看到多了一行"nan"


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Ticket,Fare,Cabin,Name_nan,Sex_female,Sex_male,Sex_nan,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,1,0,3,22.0,1,0,A/5 21171,7.2500,NaN,1,0,1,0,0,0,1,0
1,2,1,1,38.0,1,0,PC 17599,71.2833,C85,1,1,0,0,1,0,0,0
2,3,1,3,26.0,0,0,STON/O2. 3101282,7.9250,NaN,1,1,0,0,0,0,1,0
3,4,1,1,35.0,1,0,113803,53.1000,C123,1,1,0,0,0,0,1,0
4,5,0,3,35.0,0,0,373450,8.0500,NaN,1,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,27.0,0,0,211536,13.0000,NaN,1,0,1,0,0,0,1,0
887,888,1,1,19.0,0,0,112053,30.0000,B42,1,1,0,0,0,0,1,0
888,889,0,3,28.0,1,2,W./C. 6607,23.4500,NaN,1,1,0,0,0,0,1,0
889,890,1,1,26.0,0,0,111369,30.0000,C148,1,0,1,0,1,0,0,0


In [14]:
# 捨棄掉不需要的欄位

x_train_notdrop = pd.get_dummies(train_df, columns=["Name", "Sex", "Embarked"],
                                 dummy_na=True)

x_test_notdrop = pd.get_dummies(test_df, columns=["Name", "Sex", "Embarked"],
                                dummy_na=True)

# pandas.DataFrame.drop

axis 0是橫的，1是直的


In [15]:
# 把資料做最後命名

x_train = x_train_notdrop.drop(["PassengerId", "Survived", "Cabin", "Ticket"], 
                               axis=1)

y_train = x_train_notdrop["Survived"]

x_test = x_test_notdrop.drop(["PassengerId", "Cabin", "Ticket"], 
                             axis=1)

testid = x_test_notdrop["PassengerId"]

# Ensemble(組合)

## 1.Boosting(連續)
<br />
AdaBoost、XGBoost<br />
<br />
把之前錯過的條件，權重提高，最後練出一個最好的<br />
缺點是分類器要一個一個訓練，浪費時間，太依賴前一個的成果<br />
無法發揮多核電腦的效能

## 2.Bagging(平行)
<br />
一次用多種演算法去訓練分類器<br />
但是其中的初始資料要有點不一樣<br />

### 隨機森林"Random Forests"
<br />
在創造分類器的時候"放棄"一點資料，是其初始資料不一像<br />

### 隨機參數


In [22]:
# Grid Search>>>對每一組參數都做一次"Cross Validation"

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

params = {
    "n_estimators":range(25, 40) , # 20~110開始，10個一跳
    "max_depth":range(6, 11)
}

clf = RandomForestClassifier()
cv = GridSearchCV(clf, params, cv=10, n_jobs=4)
cv.fit(x_train, y_train)
print(cv.best_params_)
print(cv.best_score_)

'''
這裡試三次的結果

{'max_depth': 8, 'n_estimators': 50}
0.8417727840199751

{'max_depth': 8, 'n_estimators': 100}
0.8417727840199749

{'max_depth': 8, 'n_estimators': 50}
0.8417977528089887

之後慢慢調參數，縮小範圍直到一次跳一筆

確定結果之後下去改參數
'''

{'max_depth': 8, 'n_estimators': 50}
0.8417977528089887


In [23]:
# RandomForestClassifier
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
# 參數選擇
# 把資料拆10次算10次

import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=26, max_depth=8)  # 參數調整一次調一個就好
score = cross_val_score(clf, x_train, y_train, cv=10, n_jobs=4)
print(score)
print(np.average(score))

[0.8        0.82022472 0.75280899 0.86516854 0.92134831 0.83146067
 0.84269663 0.78651685 0.86516854 0.83146067]
0.8316853932584269


In [24]:
clf = RandomForestClassifier(n_estimators=26, max_depth=8)
clf.fit(x_train, y_train)
pre = clf.predict(x_test)
result_df = pd.DataFrame({
    "PassengerId":testid, 
    "Survived":pre
})
result_df.to_csv("rf.csv", index=False, encoding="utf-8")
result_df

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
